# Week3: Linear Programming
## Kevin Obote - 190696

This notebook demonstrates the solution to three different optimization problems using linear programming with `scipy.optimize.linprog`:

1. Transportation Problem: Optimal Shipping Plan
2. Manufacturing Problem: Maximizing Profit
3. Manufacturing Problem: Minimizing Production Cost

In [2]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

## 1. Transportation Problem: Optimal Shipping Plan

This problem involves optimizing the shipment of goods from three warehouses to four retail stores while minimizing transportation costs.

In [ ]:
# Approach 1:
# Cost matrix (warehouses × stores)
c = [
    4, 3, 6, 5,  # Costs from W1 to S1, S2, S3, S4
    2, 5, 3, 4,  # Costs from W2 to S1, S2, S3, S4
    7, 6, 4, 3   # Costs from W3 to S1, S2, S3, S4
]

# Supply constraints (≤)
A_supply = [
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],  # W1 supply
    [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],  # W2 supply
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]   # W3 supply
]
b_supply = [250, 300, 400]  # Supply limits

# Demand constraints (=)
A_demand = [
    [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],  # S1 demand
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],  # S2 demand
    [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],  # S3 demand
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]   # S4 demand
]
b_demand = [200, 200, 250, 300]  # Demand requirements

# Solve the transportation problem
result = linprog(
    c=c,
    A_ub=A_supply,
    b_ub=b_supply,
    A_eq=A_demand,
    b_eq=b_demand,
    method='highs'
)

# Format and display results
print("\nOptimal Transportation Plan:")
solution_matrix = result.x.reshape(3, 4)
df_solution = pd.DataFrame(
    solution_matrix,
    index=['W1', 'W2', 'W3'],
    columns=['S1', 'S2', 'S3', 'S4']
)
print(df_solution)
print(f"\nMinimum Total Cost: ${result.fun:.2f}")


Optimal Transportation Plan:
       S1     S2     S3     S4
W1   50.0  200.0    0.0    0.0
W2  150.0    0.0  150.0    0.0
W3    0.0    0.0  100.0  300.0

Minimum Total Cost: $2850.00


In [ ]:
# Approach 2:
def solve_transportation_problem():
    # Cost matrix (warehouses × stores)
    costs = np.array([
        [4, 3, 6, 5],  # W1 to S1,S2,S3,S4
        [2, 5, 3, 4],  # W2 to S1,S2,S3,S4
        [7, 6, 4, 3]   # W3 to S1,S2,S3,S4
    ])

    # Supply constraints (warehouses)
    supply = np.array([250, 300, 400])

    # Demand constraints (stores)
    demand = np.array([200, 200, 250, 300])

    # Number of warehouses and stores
    num_warehouses = len(supply)
    num_stores = len(demand)

    # Flatten cost matrix for linprog
    c = costs.flatten()

    # Create supply constraint matrix
    A_supply = np.zeros((num_warehouses, num_warehouses * num_stores))
    for i in range(num_warehouses):
        A_supply[i, i*num_stores:(i+1)*num_stores] = 1

    # Create demand constraint matrix
    A_demand = np.zeros((num_stores, num_warehouses * num_stores))
    for j in range(num_stores):
        A_demand[j, j::num_stores] = 1

    # Combine constraints
    A_eq = A_demand
    b_eq = demand
    A_ub = A_supply
    b_ub = supply

    # Set bounds for variables (non-negative)
    bounds = [(0, None) for _ in range(num_warehouses * num_stores)]

    # Solve the optimization problem
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, 
                    bounds=bounds, method='highs')

    if result.success:
        # Reshape solution into matrix form
        solution = result.x.reshape(num_warehouses, num_stores)
        
        print("\nOptimal Transportation Plan:")
        print("-" * 50)
        stores = ['S1', 'S2', 'S3', 'S4']
        warehouses = ['W1', 'W2', 'W3']
        
        # Print header
        print(f"{'From/To':8}", end='')
        for store in stores:
            print(f"{store:>10}", end='')
        print()
        print("-" * 50)
        
        # Print transportation quantities
        for i, warehouse in enumerate(warehouses):
            print(f"{warehouse:8}", end='')
            for j in range(num_stores):
                print(f"{solution[i,j]:10.1f}", end='')
            print()
        
        print("\nOptimization Results:")
        print(f"Total Transportation Cost: ${result.fun:.2f}")
        
        # Print supply utilization
        print("\nSupply Utilization:")
        for i, warehouse in enumerate(warehouses):
            used = sum(solution[i,:])
            print(f"{warehouse}: {used:.1f}/{supply[i]} units")
        
        # Print demand satisfaction
        print("\nDemand Satisfaction:")
        for j, store in enumerate(stores):
            received = sum(solution[:,j])
            print(f"{store}: {received:.1f}/{demand[j]} units")
            
    else:
        print("Optimization failed:")
        print(result.message)

if __name__ == "__main__":
    solve_transportation_problem()



Optimal Transportation Plan:
--------------------------------------------------
From/To         S1        S2        S3        S4
--------------------------------------------------
W1            50.0     200.0       0.0       0.0
W2           150.0       0.0     150.0       0.0
W3             0.0       0.0     100.0     300.0

Optimization Results:
Total Transportation Cost: $2850.00

Supply Utilization:
W1: 250.0/250 units
W2: 300.0/300 units
W3: 400.0/400 units

Demand Satisfaction:
S1: 200.0/200 units
S2: 200.0/200 units
S3: 250.0/250 units
S4: 300.0/300 units


## 2. Manufacturing Problem: Maximizing Profit

This problem involves determining the optimal production quantities for two products to maximize profit while respecting machine time constraints.

In [ ]:
# to maximize profit
c = [-50, -80]  # Negated profits for Products A and B

# Machine time constraints
A = [
    [3, 5],  # M1 hours per unit
    [2, 4]   # M2 hours per unit
]
b = [600, 500]  # Available hours

# Solve the maximization problem
result = linprog(
    c=c,
    A_ub=A,
    b_ub=b,
    method='highs'
)

# Format and display results
print("\nOptimal Production Plan:")
print(f"Product A: {result.x[0]:.2f} units")
print(f"Product B: {result.x[1]:.2f} units")
print(f"Maximum Total Profit: ${-result.fun:.2f}")

# Check resource utilization
m1_usage = 3 * result.x[0] + 5 * result.x[1]
m2_usage = 2 * result.x[0] + 4 * result.x[1]
print("\nResource Utilization:")
print(f"Machine 1: {m1_usage:.2f} hours used out of 600")
print(f"Machine 2: {m2_usage:.2f} hours used out of 500")


Optimal Production Plan:
Product A: 200.00 units
Product B: 0.00 units
Maximum Total Profit: $10000.00

Resource Utilization:
Machine 1: 600.00 hours used out of 600
Machine 2: 400.00 hours used out of 500


## 3. Manufacturing Problem: Minimizing Production Cost

This problem involves determining the optimal production quantities for chairs and tables to minimize cost while respecting resource constraints.

In [9]:
# Cost coefficients
c = [30, 50]  # Cost per unit for chairs and tables

# Resource constraints
A = [
    [5, 8],  # Wood required (cubic ft)
    [2, 3]   # Labor required (hours)
]
b = [800, 300]  # Available resources

# Solve the minimization problem
result = linprog(
    c=c,
    A_ub=A,
    b_ub=b,
    method='highs'
)

# Format and display results
print("\nOptimal Production Plan:")
print(f"Chairs: {result.x[0]:.2f} units")
print(f"Tables: {result.x[1]:.2f} units")
print(f"Minimum Total Cost: ${result.fun:.2f}")

# Check resource utilization
wood_usage = 5 * result.x[0] + 8 * result.x[1]
labor_usage = 2 * result.x[0] + 3 * result.x[1]
print("\nResource Utilization:")
print(f"Wood: {wood_usage:.2f} cubic ft used out of 800")
print(f"Labor: {labor_usage:.2f} hours used out of 300")


Optimal Production Plan:
Chairs: 0.00 units
Tables: 0.00 units
Minimum Total Cost: $0.00

Resource Utilization:
Wood: 0.00 cubic ft used out of 800
Labor: 0.00 hours used out of 300
